# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
# Create our features
X = df.drop(columns="loan_status")

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(X['homeowner'])
X['homeowner'] = le.transform(X['homeowner'])

# Create our target
y = df["loan_status"]

In [5]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [6]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)
X_train.shape

(58152, 8)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [8]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [10]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression(solver='lbfgs', random_state=1)
model_LR.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_LR = model_LR.predict(X_test)
balanced_accuracy_score(y_test, y_pred_LR)

0.9543211898288821

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_LR)

array([[  571,    54],
       [   93, 18666]], dtype=int64)

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_LR))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [15]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled_NRO, y_resampled_NRO = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_NRO)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [16]:
# Train the Logistic Regression model using the resampled data
model_NRO = LogisticRegression(solver='lbfgs', random_state=1)
model_NRO.fit(X_resampled_NRO, y_resampled_NRO)

LogisticRegression(random_state=1)

In [17]:
# Calculated the balanced accuracy score
y_pred_NRO = model_NRO.predict(X_test)
balanced_accuracy_score(y_test, y_pred_NRO)

0.9948013433551894

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_NRO)

array([[  622,     3],
       [  105, 18654]], dtype=int64)

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_NRO))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [20]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled_SMOTE, y_resampled_SMOTE = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
Counter(y_resampled_SMOTE)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [21]:
# Train the Logistic Regression model using the resampled data
model_SMOTE = LogisticRegression(solver='lbfgs', random_state=1)
model_SMOTE.fit(X_resampled_SMOTE, y_resampled_SMOTE)

LogisticRegression(random_state=1)

In [22]:
# Calculated the balanced accuracy score
y_pred_SMOTE = model_SMOTE.predict(X_test)
balanced_accuracy_score(y_test, y_pred_SMOTE)

0.9948279972279972

In [23]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_SMOTE)

array([[  622,     3],
       [  104, 18655]], dtype=int64)

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_SMOTE))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [25]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled_CC, y_resampled_CC = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_CC)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [26]:
# Train the Logistic Regression model using the resampled data
model_CC = LogisticRegression(solver='lbfgs', random_state=1)
model_CC.fit(X_resampled_CC, y_resampled_CC)

LogisticRegression(random_state=1)

In [27]:
# Calculate the balanced accuracy score
y_pred_CC = model_CC.predict(X_test)
balanced_accuracy_score(y_test, y_pred_CC)

0.9836813049736126

In [28]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_CC)

array([[  608,    17],
       [  102, 18657]], dtype=int64)

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_CC))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.97       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled_comb, y_resampled_comb = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_comb)

Counter({'high_risk': 55513, 'low_risk': 55930})

In [31]:
# Train the Logistic Regression model using the resampled data
model_comb = LogisticRegression(solver='lbfgs', random_state=1)
model_comb.fit(X_resampled_comb, y_resampled_comb)

LogisticRegression(random_state=1)

In [32]:
# Calculate the balanced accuracy score
y_pred_comb = model_comb.predict(X_test)
balanced_accuracy_score(y_test, y_pred_comb)

0.994748035609574

In [33]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_comb)

array([[  622,     3],
       [  107, 18652]], dtype=int64)

In [34]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_comb))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   SMOTE & naive oversampling both had the best balanced accuracy score with 0.9948.

2. Which model had the best recall score?

   SMOTE & naive oversampling and SMOTEEN [combination sampling] had the best recall scores with 0.99 and 1 in both low and high risk. All models have          pretty much the same average recall score though.

3. Which model had the best geometric mean score?

   Oversampling again, and combination as well- both with 0.99 geometric mean score.
